# `clean_ml()`: prepare data for ML with one line of code

## Motivation
Preparing data for Machine Learning is a difficult, tedious, and time consuming process. In order to mitigate these difficulties, we propose the function `clean_ml()` which is designed to transform an arbitrary tabular dataset into a format that's suitable for a typical ML application.

## Features
The function `clean_ml()` does the following alterations to the data frame:

1. **Reformat column names**. This feature follows the R library Janitor's `clean_names()` [function](http://sfirke.github.io/janitor/articles/janitor.html#clean-data-frame-names-with-clean_names). It applies the following alterations to the column names:

 a. Parses letter cases and separators to a consistent format snake_case <br>
 b. Handles special characters and spaces, including transliterating characters like œ to oe.<br>
 c. Appends numbers to duplicated names <br>
 d. Converts "%" to "percent" and "#" to "number" to retain meaning <br>
 e. Spacing (or lack thereof) around numbers is preserved <br>
 
2. **Drop rows with missing values**. The user can optionally drop all rows that contain missing values.

3. **Missing value imputation**. Impute the **mode** for categorical variables and **mean** for numerical variables to replace missing values and infinities.

4. **Drop uninformative columns**. Drop columns satisfying any of the following criteria:
 
 a. The column contains over 30 unique nominal values<br>
 b. More than 70% of the column's values are missing<br>
 c. There is a value that accounts for more than 70% of the column<br>
 d. A column contains integers and over 99% of the values are unique (likely an id column)<br>
 
5. **Transform numerical columns**. Either *standard_scaler* (default), or *minmax_scaler* applied to all numerical columns. <br>
    Standard-scaler formula:
    $$x' = \frac{x - mean(x)}{std(x)}$$
    Minmax-scaler formula:
    $$x' = \frac{x - min(x)}{max(x) - min(x)}$$
    
6. **Dummy encode categorical variables**.

7. **Downcast**. The memory unit should be appropriate and as small as possible for each column in the data frame.

# The function `clean_ml()`

In [ ]:
def clean_ml(
    df: Union[pd.DataFrame, dd.DataFrame],
    handle_missing_values: str = "impute", # or "drop"
    numerical_transform: str = "standard_scaler", # or "minmax_scaler", None
#     one_hot_enocde: bool = True,
    max_nominal_values: int = 30, # drop columns with more than 30 unique categorical values
    max_missing_values_percent: float = 70,
    max_repeated_value_percent: float = 70,
    max_unique_integers_percent: float = 99,
#     drop_columns_with = {}
    include_components = {"onehot": True, }
    exclude_components = {}
    
)

## Parameters

* `df`: Union[*pandas.DataFrame, dask.DataFrame*] &mdash; the data frame to be transformed 
* `handle_missing_values`: str, *\"impute\" or \"drop\"*, (*\"impute\"* by default) &mdash; how to deal with missing values
* `numerical_transformation`: str, *\"standard_scaler\", \"minmax_scaler\"*, or *None*, (*\"standard_scaler\"* by default) &mdash; how to transform numerical columns
* `max_unique_nominal_values`: int, default 30 &mdash; the maximum number of unique nominal values in a column
* `max_missing_values_percent`: float, default 70 &mdash; the maximum percent of values that can be missing from a column
* `max_repeated_value_percent`: float, default 70 &mdash; the maximum percent a value can repeat in a column
* `max_unique_integers_percent`: float, default 99 &mdash; the maximum percent of unique values in an integer column


## Returns

The cleaned data frame in the form (pandas or Dask) that it was passed to `clean_ml()`

## Console Output

Text output detailing what alterations have been made to the dataframe

## Example

Consider the following dataset, and pass it to `clean_ml()`

In [7]:
import pandas as pd
df = pd.DataFrame({
    "MissingNumericaL": [1, np.nan, 2, np.nan, 3],
    "MISSING CATEGORICAL": ["dog", "cat", "cat", np.nan, "cat"],
    "Too Many Missing": [np.nan, np.nan, np.nan, "hello", np.nan],
    "too many repeated values": ["dog", "dog", "dog", "dog", "cat"],
})
clean_ml(df)

,MissingNumerical,MISSING CATEGORICAL,too many unique categorical values,Too Many Missing,too many repeated values
0,1.0,dog,red,NaN,dog
1,NaN,cat,orange,NaN,dog
2,2.0,cat,yellow,NaN,dog
3,NaN,NaN,green,hello,dog
4,3.0,cat,blue,NaN,cat


The dataframe after cleaning is

In [11]:
df = pd.DataFrame({
    "missing_numerical": [-1.414214, 0, 0, 0, 1.414214],
    "missing_categorical_cat": [0, 1, 1, 1, 1],
    "missing_categorical_dog": [1, 0, 0, 0, 0],
})
df

,missing_numerical,missing_categorical_cat,missing_categorical_dog
0,-1.414214,0,1
1,0.000000,1,0
2,0.000000,1,0
3,0.000000,1,0
4,1.414214,1,0


And the **console output** is

The following column names were reformatted<br>
&emsp;Missing Numerical -> missing_numerical<br>
&emsp;MISSING CATEGORICAL -> missing_categorical
    
Imputation:<br>
&emsp;Column *missing_numerical* imputed the value 2 for 2 rows (40% of data)<br>
&emsp;Column *missing_categorical* imputed the value "dog" for 1 tuple (20% of data)

Transformations:<br>
&emsp;Column *missing_numerical* was transformed with standard scaler<br>
&emsp;Column *missing_categorical* was one-hot encoded

Irrelevant columns dropped:<br>
&emsp;Column *Too Many Missing* was dropped because of 80% of values are missing<br>
&emsp;Column *too many repeated values* was dropped because the value "dog" accounts for 80% of the column


# Related work


## [datacleaner](https://github.com/rhiever/datacleaner)
1. Drop any rows with missing values (default False)
2. Impute missing values: mode for categorical, mean for continuous
3. Encode categorical variables: default LabelEncoder (can pass an sklearn encoder)

## RapidMiner
1. Define the target column (so it is ignored from cleaning), ignore if doesn't exist
2. Remove low quality columns
3. Change type to all numerical (dummy), or all categorical (bin)
4. Specify PCA or normalization

## [PyCaret](https://pycaret.org/classification/)
Takes as input a dataframe and target column
1. default *constant* ("not available") categorical imputation, other option "mode"
2. default *mean* numerical imputation, other option "median"
3. high cardinality transformation *frequency*, other "clustering"

## [vtreat](https://github.com/WinVector/pyvtreat)
1. Define the target column (it can't contain missing values)
2. Statistical approach to dealing with missing values
3. For categorical encoding:
    a) impact (target) encoding
4. Use a calibration set to determine whether the encoding introduces bias